In [1]:
import numpy as np
import pandas as pd
import gc
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

In [2]:
df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl")

In [3]:
[col for col in df.columns if "LAST_" not in col]

['GAME_ID',
 'TEAM_ID_HOME',
 'TEAM_ID_AWAY',
 'PLUS_MINUS',
 'GAME_DATE',
 'TEAM_ID_H',
 'TEAM_ID_A']

In [4]:
len([col for col in df.columns if "_H" in col])

362

In [5]:
len([col for col in df.columns if ("_A" in col) & ("_H" not in col)])

362

In [6]:
set(df.columns) - set([col for col in df.columns if "_H" in col]) - set([col for col in df.columns if ("_A" in col) & ("_H" not in col)])

{'GAME_DATE', 'GAME_ID', 'PLUS_MINUS'}

In [7]:
df["TEAM_ID_AWAY"] = df["TEAM_ID_AWAY"].astype("category")
df["TEAM_ID_HOME"] = df["TEAM_ID_HOME"].astype("category")

In [8]:
np.percentile(df.loc[:, "PLUS_MINUS"].values, [15, 50, 75]), np.mean(df.loc[:, "PLUS_MINUS"].values)

(array([-11.,   4.,  11.]), 2.690338713343942)

In [9]:
column = ["_".join(col.split("_")[2:-1]) for col in df.columns if ("LAST_1_" in col)&("_H" in col)]

In [ ]:
# 修改極大或極小
df.loc[df[df.loc[:,"PLUS_MINUS"]>=11].index,"PLUS_MINUS"] = 11
df.loc[df[df.loc[:,"PLUS_MINUS"]<=-11].index,"PLUS_MINUS"] = -11

In [10]:
new_col = []
for col_i in column:
    new_col += [col for col in df.columns if col_i in col]

In [11]:
new_col = list(set(df.columns)-set(new_col))+list(new_col)

In [12]:
len(set(new_col))

727

In [13]:
df = df[new_col]

In [14]:
df.head()

,PLUS_MINUS,TEAM_ID_AWAY,TEAM_ID_A,GAME_ID,GAME_DATE,TEAM_ID_H,TEAM_ID_HOME,LAST_1_GAME_FGM_H,LAST_2_GAME_FGM_H,LAST_3_GAME_FGM_H,...,LAST_1_GAME_PLAYER_PIE_A,LAST_2_GAME_PLAYER_PIE_A,LAST_3_GAME_PLAYER_PIE_A,LAST_4_GAME_PLAYER_PIE_A,LAST_5_GAME_PLAYER_PIE_A,LAST_6_GAME_PLAYER_PIE_A,LAST_7_GAME_PLAYER_PIE_A,LAST_8_GAME_PLAYER_PIE_A,LAST_9_GAME_PLAYER_PIE_A,LAST_10_GAME_PLAYER_PIE_A
0,-39.0,1610612741,1610612741,0021100222,2012-01-20,1610612739,1610612739,35.0,37.0,35.333333,...,0.099561,0.098817,0.088188,0.102924,0.100449,0.108826,0.090536,0.088891,0.089388,0.085876
1,15.0,1610612751,1610612751,0021100244,2012-01-23,1610612741,1610612741,40.0,43.0,44.333333,...,0.090257,0.091350,0.081796,0.077719,0.077584,0.078040,0.074292,0.072650,0.073911,0.069259
2,12.0,1610612758,1610612758,0021100250,2012-01-23,1610612757,1610612757,34.0,33.5,34.000000,...,0.091012,0.083085,0.073050,0.060630,0.063579,0.061998,0.064544,0.067435,0.067592,0.065412
3,7.0,1610612739,1610612739,0021100254,2012-01-24,1610612748,1610612748,25.0,34.5,35.666667,...,0.044851,0.044384,0.059031,0.067610,0.071996,0.067467,0.069797,0.076624,0.075287,0.078071
4,-3.0,1610612761,1610612761,0021100255,2012-01-24,1610612756,1610612756,31.0,31.0,32.666667,...,0.105249,0.091096,0.099990,0.089473,0.099120,0.101186,0.099608,0.100829,0.093281,0.090670


In [15]:
df_train = df[df.GAME_ID<"0021700000"].reset_index(drop=True)
df_val = df[(df.GAME_ID<"0021800000")&(df.GAME_ID>"0021700000")].reset_index(drop=True)

In [16]:
x_train = df_train.loc[:, [col for col in df_train.columns if (col not in ["GAME_ID","GAME_DATE","PLUS_MINUS"])&
                          ("TEAM_ID" not in col)]]
x_val = df_val.loc[:, [col for col in df_val.columns if (col not in ["GAME_ID","GAME_DATE","PLUS_MINUS"])&
                        ("TEAM_ID" not in col)]]

In [17]:
y_train = df_train.loc[:, "PLUS_MINUS"]
y_val = df_val.loc[:, "PLUS_MINUS"]

In [18]:
train_dataset = lgb.Dataset(x_train, y_train)
test_dataset = lgb.Dataset(x_val, y_val)

In [19]:
def lgb_eval(learning_rate, num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    evals_result = {}
    params = {'application':'regression_l2','num_iterations': 1000, 'early_stopping_round':100, 'metric':'l2'}
    
    params["num_leaves"] = int(round(num_leaves))
    params["learning_rate"] = max(learning_rate, 0)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    result = lgb.train(train_set=train_dataset,valid_sets=[train_dataset, test_dataset] ,evals_result=evals_result,
                       params=params,verbose_eval =200)
    return -min(evals_result["valid_1"]["l2"])

lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.001, 0.1),
                                        'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.5, 1),
                                        'max_depth': (4, 9),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=7)

In [20]:
%%time
lgbBO.maximize(init_points=25, n_iter=75)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------


/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:107: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[31]	training's l2: 107.093	valid_1's l2: 168.675
|  1        | -168.7    |  0.5382   |  0.7239   |  2.192    |  2.17     |  0.09782  |  6.692    |  27.55    |  0.008133 |  29.64    |
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 131.685	valid_1's l2: 169.483
[400]	training's l2: 117.074	valid_1's l2: 167.862
Early stopping, best iteration is:
[424]	training's l2: 115.963	valid_1's l2: 167.777
|  2        | -167.8    |  0.7499   |  0.6434   |  4.019    |  1.143    |  0.007528 |  5.441    |  45.93    |  0.02213  |  33.49    |
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 96.0262	valid_1's l2: 167.493
Early stopping, best iteration is:
[104]	training's l2: 116.441	valid_1's l2: 166.7
|  3        | -166.7    |  0.9656   |  0.1199   |  3.003    |  2.85     |  0.0238   |  6.742    |  45.91    |  0.01418  |  34.99    |
T

Early stopping, best iteration is:
[160]	training's l2: 99.3094	valid_1's l2: 167.151
|  21       | -167.2    |  0.7675   |  0.3536   |  3.685    |  0.4806   |  0.02006  |  5.773    |  22.03    |  0.02142  |  43.29    |
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[70]	training's l2: 117.377	valid_1's l2: 165.672
|  22       | -165.7    |  0.914    |  0.1855   |  1.847    |  0.698    |  0.04566  |  5.382    |  27.58    |  0.09234  |  32.03    |
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[21]	training's l2: 121.192	valid_1's l2: 170.254
|  23       | -170.3    |  0.8251   |  0.5765   |  3.76     |  0.185    |  0.07474  |  8.731    |  32.16    |  0.02947  |  38.12    |
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[43]	training's l2: 95.545	valid_1's l2: 170.255
|  24       | -170.3    |  0.856    |  0.6252   |  0.7347   |  2.92  

/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:107: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[90]	training's l2: 117.272	valid_1's l2: 165.484
|  98       | -165.5    |  0.7      |  0.1      |  0.573    |  2.186    |  0.03737  |  7.4      |  48.03    |  0.08172  |  25.06    |


/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:107: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[36]	training's l2: 120.258	valid_1's l2: 167.054
|  99       | -167.1    |  0.5767   |  0.1233   |  0.1782   |  0.9062   |  0.08046  |  4.581    |  5.786    |  0.003666 |  38.21    |


/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:107: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[32]	training's l2: 118.145	valid_1's l2: 168.801
|  100      | -168.8    |  0.8526   |  0.3161   |  0.2489   |  1.24     |  0.08615  |  4.551    |  5.949    |  0.01737  |  38.78    |
CPU times: user 25min 47s, sys: 4min 38s, total: 30min 26s
Wall time: 9min 50s


In [23]:
 lgbBO.res[np.argmax([i["target"] for i in lgbBO.res])]

{'params': {'bagging_fraction': 0.8706187283316725,
  'feature_fraction': 0.1,
  'lambda_l1': 0.00019934555725044367,
  'lambda_l2': 1.0830676860368973,
  'learning_rate': 0.010947383797439775,
  'max_depth': 4.673263585901005,
  'min_child_weight': 6.177336281660579,
  'min_split_gain': 0.035035585082948205,
  'num_leaves': 38.47712277120209},
 'target': -164.61567653589103}

In [21]:
best_params = lgbBO.res[np.argmax([i["target"] for i in lgbBO.res])]["params"]

In [22]:
best_params

{'bagging_fraction': 0.8706187283316725,
 'feature_fraction': 0.1,
 'lambda_l1': 0.00019934555725044367,
 'lambda_l2': 1.0830676860368973,
 'learning_rate': 0.010947383797439775,
 'max_depth': 4.673263585901005,
 'min_child_weight': 6.177336281660579,
 'min_split_gain': 0.035035585082948205,
 'num_leaves': 38.47712277120209}

In [ ]:
best_params["num_leaves"] = int(round(best_params["num_leaves"]))
best_params["max_depth"] = int(round(best_params["max_depth"]))

In [ ]:
best_params.update({'num_iterations': 1000,
                    'early_stopping_round':100,
                    "metric": "mean_squared_error"})

In [ ]:
best_params

In [ ]:
import pickle

In [ ]:
pickle.dump(best_params, open("/Users/chienan/Pycon/github/SportLottery/model/model_params.pkl","wb"),protocol=-1)

In [ ]:
%%time 
print("start training...")
model = lgb.train(train_set=train_dataset,
                  valid_sets=[train_dataset, test_dataset] ,
                  verbose_eval=20,
                  params=best_params,
                  evals_result=evals_result)

In [ ]:
# model.save_model('/Users/chienan/Pycon/github/SportLottery/model/model_agg_v1.txt')

In [ ]:
lgb.plot_importance(booster=model,max_num_features=20,importance_type="split")
plt.show()

In [ ]:
sum(model.predict(x_val)>0.5), sum(y_val)

In [ ]:
# accuracy 
# 0.6365638766519823
sum((model.predict(x_val)>0.5).astype(int)==y_val)/len(y_val)

In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score,precision_recall_curve

In [ ]:
f1_score(y_pred=(model.predict(x_val)>0.5).astype(int),y_true=y_val)

In [ ]:
precision_score(y_pred=(model.predict(x_val)>0.5).astype(int),y_true=y_val)

In [ ]:
recall_score(y_pred=(model.predict(x_val)>0.5).astype(int),y_true=y_val)

In [ ]:
precision_recall_curve(probas_pred=model.predict(x_val),y_true=y_val)

In [ ]:
data_lst = [col for col in x_train.columns[model.feature_importance().argsort()][::-1] if "TEAM_ID" not in col]

In [ ]:
data_lst = [col for col in x_train.columns[model.feature_importance().argsort()][::-1] if "TEAM_ID" not in col]
lst = []

while data_lst:
    add_item = list(set([data_lst[0] for i in column if (i in data_lst[0])&("_H" in data_lst[0])] + 
                        [data_lst[0] for i in column if (i in data_lst[0])&("_A" in data_lst[0])]))
    lst += add_item
    data_lst = [col for col in data_lst if "_".join(lst[-1].split("_")[3:]) != "_".join(col.split("_")[3:])]

In [ ]:
x_train = df_train.loc[:,lst]
x_test = df_test.loc[:,lst]

In [ ]:
%%time
print("start training...")
model_v2 = lgb.train(train_set=train_dataset, params=param)

In [ ]:
lgb.plot_importance(booster=model_v2,max_num_features=20,importance_type="split")
plt.show()

In [ ]:
sum((model_v2.predict(x_test)>0.4).astype(int)==y_test)/len(y_test)

In [ ]:
model_v2.predict(x_test)

In [ ]:
sum(y_test[model_v2.predict(x_test)>0.4])/sum((model_v2.predict(x_test)>0.4))

In [ ]:
sum(model_v2.predict(x_test)[y_test.astype(bool)]>0.4)/sum(y_test)

In [ ]:
sum(model_v2.predict(x_test)>0.4)

In [ ]:
sum(y_test),len(y_test)